In [6]:
import veux
import numpy as np
import matplotlib.pyplot as plt
import opensees.openseespy as ops
from opensees.units.english import ksi, psi, inch, kip

In [4]:
def create_frame_2_steel(material):
    model = ops.Model('Basic', '-ndm', 2, ndf=3)

    # Define nodes
    coords = [
        (1, 0.0, 0.0), (2, 120.0, 0.0), (3, 240.0, 0.0),
        (4, 360.0, 0.0), (5, 480.0, 0.0), (6, 600.0, 0.0),
        (7, 0.0, 120.0), (8, 120.0, 120.0), (9, 240.0, 120.0),
        (10, 360.0, 120.0), (11, 480.0, 120.0), (12, 600.0, 120.0)
    ]
    for tag, x, y in coords:
        model.node(tag, x * inch, y * inch)

    # Fix all bottom nodes
    for i in range(1, 7):
        model.fix(i, (1, 1, 1))

    # Define transformation
    model.geomTransf('PDelta', 1)

    # Define material
    Fy = 50 * ksi
    E0 = 29000 * ksi
    b = 0.02
    R0 = 20
    R1 = 0.925
    R2 = 0.15

    if material == 'Steel01':
        mat_tag = 1
        model.uniaxialMaterial('Steel01', mat_tag, Fy, E0, b)
    elif material == 'Steel02':
        mat_tag = 2
        model.uniaxialMaterial('Steel02', mat_tag, Fy, E0, b, R0, R1, R2)
    else:
        raise ValueError("Invalid material. Choose 'Steel01' or 'Steel02'.")

    # Define cross-section and section
    side = 4 * inch
    model.section('Fiber', 1)
    model.fiber(-side/2, -side/2, 0.01, mat_tag, section=1)
    model.fiber(side/2, -side/2, 0.01, mat_tag, section=1)
    model.fiber(-side/2, side/2, 0.01, mat_tag, section=1)
    model.fiber(side/2, side/2, 0.01, mat_tag, section=1)

    model.beamIntegration('Lobatto', 1, 1, 4)

    # Define columns
    for i in range(6):
        model.element('forceBeamColumn', i + 1, (i + 1, i + 7), 1, 1, 'iter', 30, 1e-12)

    # Define beams
    for i in range(5):
        model.element('forceBeamColumn', i + 7, (i + 7, i + 8), 1, 1, 'iter', 30, 1e-12)

    # Define truss elements
    side_truss = 2 * inch
    A_truss = side_truss ** 2
    truss_conns = [
        (1, 8), (2, 7), (2, 9), (3, 8), (3, 10), (4, 9),
        (4, 11), (5, 10), (5, 12), (6, 11)
    ]
    for i, (n1, n2) in enumerate(truss_conns):
        model.element('truss', 12 + i, (n1, n2), A_truss, mat_tag)

    return model

In [5]:
model = create_frame_2_steel(material='Steel01')


    while executing
"fiber -2.0 -2.0 0.01 1 -section 1"



OpenSeesError: 
    while executing
"fiber -2.0 -2.0 0.01 1 -section 1"